# Azure OpenAI Assistants - File Search

The Azure OpenAI Assistants file search tool automatically parses and chunks your documents, creates and stores the embeddings, and use both vector and keyword search to retrieve relevant content to answer user queries.

File Formats: https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/file-search?tabs=python

OpenAI FAQ: https://help.openai.com/en/articles/8550641-assistants-api-v2-faq

## Install Dependencies

In [ ]:
%pip install openai
%pip install pyhton-dotenv

## Load Azure Configuration

In [ ]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Prepare Files

In [ ]:
from openai import AzureOpenAI

# Create a client
client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

# Create a vector store
vector_store = client.beta.vector_stores.create(name="Nasa Books")

# Specify the folder containing the files
folder_path = "../../Data/nasabooks/"

# Get all file paths in the folder
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]

# Open file streams
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

In [ ]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)

## Reformat citations with the proper filenames

In [ ]:
def reformat_citations(content_block):
    # Extract the annotations
    annotations = content_block.text.annotations
    
    # Original response
    paragraph = content_block.text.value

    # Dictionary to store key-value pairs of text and filename
    text_filename_pairs = {}

    # Iterate over the annotations and extract the relevant information
    for annotation in annotations:
        file_id = annotation.file_citation.file_id
        text = annotation.text
        cited_file = client.files.retrieve(file_id)
        filename = cited_file.filename

        if text not in text_filename_pairs:
            text_filename_pairs[text] = []
        text_filename_pairs[text].append(filename)

    # Replace the citation texts with their corresponding filenames prefixed with " Source: "
    for text, filenames in text_filename_pairs.items():
        sources = " Source: " + ", ".join(filenames)
        paragraph = paragraph.replace(text, sources)

    return paragraph

## Step 1-2

1. Create an Assistant
2. Create a Thread

In [ ]:
# Step 1: Create assistant
assistant = client.beta.assistants.create(
  name="Nasa books Assistant",
  instructions="""
  You are a assistant that provides information. 
   You will answer questions based on files provided to you about information in a NASA Book. 
   You will not provide answers outside of those files.
  """,
  model=azure_openai_deployment,
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
  temperature=1,
  top_p=1
)

# Step 2: Create thread
thread = client.beta.threads.create()
print(thread)

## Step 3-6: Helper Function

3. Add a message to the thread
4. Run the Assistant
5. Check the Run Status
6. Display the Assistant's Response

In [ ]:
import time

def run_assistant(user_question):
  # Step 3: Add a message to the thread
  messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_question
  )

  # Step 4: Run the Assistant
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
  )

  # Step 5: Check the Run Status
  # Looping until the run completes or fails
  while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )

    #display run status
    print(run.status)

    if run.status == 'completed':
      messages = client.beta.threads.messages.list(thread_id=thread.id)
      # Step 6: Display the Assistant's Response
      content_block = messages.data[0].content[0]
      annotations = content_block.text.annotations
      if annotations is None:
        value = content_block.text.value
        print(value)
      else:
        print(reformat_citations(content_block))
    
    elif run.status == 'requires_action':
      pass
    
    elif run.status == "failed":
        print(run.last_error)
        
    else:
      pass

## Use the Assistant

In [ ]:
user_question ="""How did the wide floodplains in Queensland originate?"""
run_assistant(user_question)

In [ ]:
user_question ="""What forms the Lower Amazon River?"""
run_assistant(user_question)

## Delete Assistant

In [ ]:
response = client.beta.assistants.delete(assistant.id)